In [121]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [122]:
data = pd.read_csv('dengueKaggle.csv', sep=',')
answers = pd.read_csv('totalcases.csv', sep=',')

data['total_cases'] = answers['total_cases']

data.shape, answers.shape

((1456, 25), (1456, 4))

In [123]:
data_iq = data[data['city'] == 'iq']
data_iq['week_start_date'] = data_iq['week_start_date'].apply(pd.to_datetime)

In [124]:
discarded_iq_data = data_iq[data_iq['week_start_date'] < '2000-07-01']

data_iq = data_iq[data_iq['week_start_date'] > '2000-06-30']


data_iq = data_iq.drop(columns=['city']).reset_index()

In [125]:
data_iq = data_iq[data_iq['week_start_date'] > '2007-12-31']
data_iq['week_start_date'].describe()

count                     234
unique                    234
top       2006-01-01 00:00:00
freq                        1
first     2006-01-01 00:00:00
last      2010-06-25 00:00:00
Name: week_start_date, dtype: object

In [141]:
data1 = data_iq[data_iq['week_start_date'] > '2005-12-31']
data2 = data_iq[data_iq['week_start_date'] > '2007-12-31']

In [146]:
result = pd.concat([data1, data2]).drop(columns=['index'])


In [165]:
result['precipitation_amt_mm'].describe()

count    360.000000
mean      66.738750
std       36.206721
min        0.000000
25%       41.302500
50%       62.840000
75%       89.332500
max      210.830000
Name: precipitation_amt_mm, dtype: float64

In [149]:
result.columns

Index(['year', 'weekofyear', 'week_start_date', 'ndvi_ne', 'ndvi_nw',
       'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm', 'reanalysis_air_temp_k',
       'reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k',
       'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k',
       'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm', 'total_cases'],
      dtype='object')